In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout,Dense,BatchNormalization,SimpleRNN,Embedding,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import plot_model,pad_sequences
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import math
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
with open("acc.txt", "r", encoding="utf-8") as f:
    data = f.read()
new_li = [line.strip().lower()
          for line in data.split("\n")
          if line.strip() != ""]

In [3]:
seq_len = 10
def build_sequences(text,seq_len = 5):
    token = Tokenizer()
    token.fit_on_texts(text)
    sequences = []
    for line in text:
        token_list = token.texts_to_sequences([line])[0]
        for i in range(seq_len, len(token_list)):
            sequences.append(token_list[i-seq_len:i+1])
    sequences = pad_sequences(sequences, maxlen=seq_len+1, padding="pre")
    return sequences, token

ip_sequences, token = build_sequences(new_li,seq_len)

In [4]:
x = ip_sequences[:,:-1]
y = ip_sequences[:,-1]

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=7)

In [6]:
def evaluation_of_model(model_name):
  loss = model_name.evaluate(x_test, y_test, verbose=0)
  print(f"Test Loss: {loss:.4f}")
  perplexity = math.exp(loss)
  print(f"Perplexity: {perplexity:.2f}")

In [7]:
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds)
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)
index_word = {v:k for k,v in token.word_index.items()}
def generate_text(seed_text, model, next_words=10, temperature=0.7):
    for _ in range(next_words):
        seq = token.texts_to_sequences([seed_text])[0]
        seq = pad_sequences([seq], maxlen=seq_len)
        preds = model.predict(seq, verbose=0)[0]
        next_index = sample_with_temperature(preds, temperature)
        next_word = index_word.get(next_index, "")
        seed_text += " " + next_word
    return seed_text

In [8]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=len(token.word_index)+1,output_dim = 16))
rnn_model.add(SimpleRNN(128))
rnn_model.add(Dense(len(token.word_index)+1,activation = 'softmax'))
rnn_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')
history_rnn = rnn_model.fit(x_train,y_train,epochs = 20,verbose = 0, batch_size = 64)

In [9]:
print(evaluation_of_model(rnn_model))
print(generate_text("inheritance values are",rnn_model))

Test Loss: 9.7209
Perplexity: 16662.26
None
inheritance values are often data created obtained context of a phrase schemas in


In [10]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=len(token.word_index)+1,output_dim = 100))
lstm_model.add(LSTM(300,return_sequences=True,dropout=0.2))
lstm_model.add(LSTM(300,dropout=0.2))
lstm_model.add(Dense(len(token.word_index)+1,activation = 'softmax'))
lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy')
history_lstm = lstm_model.fit(x_train,y_train,epochs = 20,batch_size = 64)

Epoch 1/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 7.5803
Epoch 2/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.8639
Epoch 3/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.7004
Epoch 4/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.5962
Epoch 5/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 6.4521
Epoch 6/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.3205
Epoch 7/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.1275
Epoch 8/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.9789
Epoch 9/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.8038
Epoch 10/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 5.6454
Epoch 11/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.5169
Epoch 12/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.3433
Epoch 13/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.1836
Epoch 14/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 4.9938
Epoch 15/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - 

In [11]:
print(evaluation_of_model(lstm_model))
print(generate_text("the centre",lstm_model))

Test Loss: 9.4513
Perplexity: 12724.68
None
the centre of the scientific html content for a topic for the


In [12]:
print(generate_text("the billionaire", lstm_model, temperature=0.3))
print(generate_text("the billionaire", lstm_model, temperature=0.7))
print(generate_text("the billionaire", lstm_model, temperature=1.1))

the billionaire of the brain’s database did his help in the sea
the billionaire country using einstein’s rule and 2004 the indian language and
the billionaire country after researchers of the iconic to grow scientific us
